This notebook provides examples to go along with the [textbook](https://underactuated.csail.mit.edu/simple_legs.html).  I recommend having both windows open, side-by-side!


In [ ]:
import mpld3
import numpy as np
from IPython.display import display
from matplotlib import pyplot as plt
from pydrake.all import (
    DiagramBuilder,
    MeshcatVisualizer,
    SceneGraph,
    Simulator,
    StartMeshcat,
)
from pydrake.examples import (
    RimlessWheel,
    RimlessWheelGeometry,
    RimlessWheelParams,
)

from underactuated import running_as_notebook


if running_as_notebook:
    mpld3.enable_notebook()

In [ ]:
# Start the visualizer (run this cell only once, each instance consumes a port)
meshcat = StartMeshcat()

# The Rimless Wheel

In [ ]:
def rimless_wheel(slope=0.08, initial_angle=0, initial_angular_velocity=5.0):
    params = RimlessWheelParams()
    params.set_slope(slope)

    builder = DiagramBuilder()
    rimless_wheel = builder.AddSystem(RimlessWheel())
    scene_graph = builder.AddSystem(SceneGraph())
    RimlessWheelGeometry.AddToBuilder(
        builder,
        rimless_wheel.get_floating_base_state_output_port(),
        params,
        scene_graph,
    )
    visualizer = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)
    meshcat.Set2dRenderMode(xmin=-2, xmax=14, ymin=-2, ymax=3)

    diagram = builder.Build()
    simulator = Simulator(diagram)

    context = simulator.get_mutable_context()

    diagram.GetMutableSubsystemContext(
        rimless_wheel, context
    ).get_numeric_parameter(0).set_slope(slope)
    context.SetAccuracy(1e-4)
    context.SetContinuousState([initial_angle, initial_angular_velocity])
    simulator.Initialize()
    visualizer.StartRecording(False)
    simulator.AdvanceTo(5.0 if running_as_notebook else 0.1)
    visualizer.PublishRecording()


rimless_wheel()

In [ ]:
rimless_wheel(initial_angular_velocity=5.0)

In [ ]:
rimless_wheel(initial_angular_velocity=10.0)

In [ ]:
rimless_wheel(initial_angular_velocity=0.95)

In [ ]:
rimless_wheel(initial_angular_velocity=-5.0)

In [ ]:
rimless_wheel(initial_angular_velocity=-4.8)

Here is a little interactive plot to allow you to visualize the trajectories of the rimless wheel as you vary the initial velocity.

In [ ]:
# TODO(russt): Port to meshcat and make it interactive, once again.


def rimless_wheel_return_map(initial_angular_velocity=1.5, duration=1.5):
    rimless_wheel = RimlessWheel()
    simulator = Simulator(rimless_wheel)
    context = simulator.get_mutable_context()
    params = context.get_numeric_parameter(0)
    qmin = params.slope() - rimless_wheel.calc_alpha(params) - 0.1
    qmax = params.slope() + rimless_wheel.calc_alpha(params) + 0.1
    qdmin = -2
    qdmax = 2

    context.SetAccuracy(1e-2)
    integrator = simulator.get_mutable_integrator()

    fig, ax = plt.subplots(figsize=(10, 6))

    # TODO(russt): make the slope interactive, too.
    def simulate(initial_angular_velocity=1.5, duration=1.5):
        rimless_wheel.SetDefaultContext(context)
        context.SetTime(0.0)
        if initial_angular_velocity >= 0:
            initial_angle = params.slope() - rimless_wheel.calc_alpha(params)
        else:
            initial_angle = params.slope() + rimless_wheel.calc_alpha(params)
        if initial_angular_velocity == 0:
            # Set double_support = True.
            context.get_mutable_abstract_state(0).set_value(True)

        context.SetContinuousState([initial_angle, initial_angular_velocity])

        integrator.StartDenseIntegration()
        simulator.Initialize()
        simulator.AdvanceTo(duration if running_as_notebook else 0.1)
        pp = integrator.StopDenseIntegration()

        return pp.vector_values(pp.get_segment_times())

    if False:
        data = simulate()
        (line,) = ax.plot(data[0, :], data[1, :], "b")
        (pt,) = ax.plot(data[0, 0], data[1, 0], "b*", markersize=12)

        def update(initial_angular_velocity):
            data = simulate(initial_angular_velocity)
            line.set_xdata(data[0, :])
            line.set_ydata(data[1, :])
            pt.set_xdata(data[0, 0])
            pt.set_ydata(data[1, 0])
            fig.canvas.draw()

        interact(
            update,
            initial_angular_velocity=widgets.FloatSlider(
                min=qdmin, max=qdmax, step=0.1, value=1.1
            ),
        )

    else:
        data = simulate(initial_angular_velocity, duration)
        ax.plot(data[0, :], data[1, :], "b")
        ax.plot(data[0, 0], data[1, 0], "b*", markersize=12)

    # Plot the energy contours.
    nq = 151
    nqd = 151
    mgl = params.mass() * params.gravity() * params.length()
    q = np.linspace(qmin, qmax, nq)
    qd = np.linspace(qdmin, qdmax, nqd)
    Q, QD = np.meshgrid(q, qd)
    Energy = 0.5 * params.mass() * params.length() ** 2 * QD**2 + mgl * np.cos(
        Q
    )
    ax.contour(
        Q,
        QD,
        Energy,
        alpha=0.5,
        linestyles="dashed",
        colors="black",
        linewidths=0.5,
    )

    ax.plot(
        params.slope() - rimless_wheel.calc_alpha(params) * np.array([1, 1]),
        np.array([0, qdmax]),
        "k--",
    )
    ax.plot(
        params.slope() - rimless_wheel.calc_alpha(params) * np.array([1, 1]),
        np.array([0, qdmin]),
        "k",
        linewidth=0.25,
    )
    ax.plot(
        params.slope() + rimless_wheel.calc_alpha(params) * np.array([1, 1]),
        np.array([0, qdmin]),
        "k--",
    )
    ax.plot(
        params.slope() + rimless_wheel.calc_alpha(params) * np.array([1, 1]),
        np.array([0, qdmax]),
        "k",
        linewidth=0.25,
    )
    ax.plot([qmin, qmax], [0, 0], "k", linewidth=0.25)
    ax.plot([0, 0], [qdmin, qdmax], "k", linewidth=0.25)
    ax.set_xlabel("theta")
    ax.set_ylabel("thetadot")
    ax.axis([qmin, qmax, qdmin, qdmax])
    ax.set_title(
        "Trajectories of the Rimless Wheel (w/ contours of " "constant energy)"
    )
    display(mpld3.display())


# Interesting angular velocities to try: 1.1, 1.5, 4, 1.0, 0.95, -5, -4.8
rimless_wheel_return_map(initial_angular_velocity=1.0, duration=5)